# Installing and Importing Required Libraries . 

In [ ]:
!pip install contractions
!pip install pyspellchecker
!pip install -U textblob

In [ ]:
!python -m textblob.download_corpora

In [ ]:
!pip install wordcloud

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk 
import string
import string
import contractions
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from sklearn.model_selection import train_test_split
import random
import os
import sys
import seaborn as sns

In [ ]:
from keras.preprocessing.text import Tokenizer, one_hot
from keras.utils import pad_sequences
from more_itertools import locate

In [ ]:
from tensorflow.keras.layers import Embedding,Dense,Dropout,Concatenate,Flatten,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization

from tensorflow.keras import Model, regularizers, optimizers
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from tensorflow import keras

# Read Data from kaggle Input files

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sys.path.append('/kaggle/input')

In [ ]:
print(stopwords.words('english'))

In [ ]:
# load the data to a pandas dataframe
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test  = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
abbreviations = pd.read_csv("/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.csv")
abrevtn_dict   = dict(zip(abbreviations.acronym, abbreviations.expansion))

In [ ]:
train

In [ ]:
train.describe()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# checking for missing values

train.isnull().sum()

In [ ]:
train.dropna(inplace=True)

In [ ]:
# checking for missing values
train.isnull().sum()

In [ ]:
# checking for missing values

test.isnull().sum()

In [ ]:
test.dropna(inplace=True)

In [ ]:
# checking for missing values
test.isnull().sum()

# Explore Dataset

In [ ]:
sentences = train["text"].tolist()

In [ ]:
sentences2 = " ".join(sentences)

In [ ]:
# To show most frequentsd words in our data

plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences2))

In [ ]:
def clear_text(text):
  text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
  text=re.sub('`',"'",text)        #Replacing apostrophe
  text=re.sub('\S*\d\S*',' ',text) #Removing Numbers
  text=re.sub('<.*?>+',' ',text)   #Removing Angular Brackets
  text=re.sub('\[.*?\]',' ',text)  #Removing Square Brackets
  text=re.sub('\n',' ',text)       #Removing '\n' character 
  text=re.sub('\*+','INSULT',text) #Replacing **** by INSULT
  return text

In [ ]:
train['text'] = train['text'].apply(clear_text)

train['text']

In [ ]:
positive = train[train["sentiment"] == "positive"]
positive

In [ ]:
negative = train[train["sentiment"] == "negative"]
negative

In [ ]:
neutral = train[train["sentiment"] == "neutral"]
neutral

In [ ]:
# To show most frequentsd words in positive data samples
pos = positive["text"].tolist()
pos2 = " ".join(pos)
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(pos2))

In [ ]:
# To show most frequentsd words in negative data samples
neg = negative["text"].tolist()
neg2 = " ".join(neg)
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(neg2))

# Data Pre-Processing

In [ ]:
port_stem = PorterStemmer()
spell = SpellChecker()


In [ ]:
def remove_hyperlinks(text):
  hyperlinkfree = re.sub('https?://\S+|www\.\S+', '', text)
  return hyperlinkfree

In [ ]:
def stemming(text):
  
  stemmed_content = text.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [ ]:
def clear_text(text):
  text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
  text=re.sub('`',"'",text)        #Replacing apostrophe
  text=re.sub('\S*\d\S*',' ',text) #Removing Numbers
  text=re.sub('<.*?>+',' ',text)   #Removing Angular Brackets
  text=re.sub('\[.*?\]',' ',text)  #Removing Square Brackets
  text=re.sub('\n',' ',text)       #Removing '\n' character 
  text=re.sub('\*+','INSULT',text) #Replacing **** by INSULT
  return text

In [ ]:
def expand_contractions(text):
    expanded_words = []   
    for word in text.split():
      # using contractions.fix to expand the shortened words
      expanded_words.append(contractions.fix(word))     
    expanded_text = contractions.fix(text)
    return expanded_text



In [ ]:
def abbrev_word(word):
    if word in abrevtn_dict.keys():
        return abrevtn_dict[word]
    else: 
        return word
    
def abbrev2_text(text):
    sentnce = [abbrev_word(word) for word in text.split()]
    text = ' '.join(sentnce)
    return text
 

In [ ]:
# to remove links from our data as it wouldn't affect our prediction or the meaning of the sentence 

train['text'] = train['text'].apply(remove_hyperlinks)
print(train['text'])

train['selected_text'] = train['selected_text'].apply(remove_hyperlinks)
print("\n\n", train['selected_text'])

In [ ]:
# to expand apprevations from the data ex : "I'd ve" would be replaced by "I would have"

train['text'] = train['text'].apply(expand_contractions)
print(train['text'])

train['selected_text'] = train['selected_text'].apply(expand_contractions)
print("\n\n",train['selected_text'])

In [ ]:
# to make the text clear by replacing apostrophe, numbers , etc.... which would make understanding words by model more easier

train['text'] = train['text'].apply(clear_text)
print(train['text'])

train['selected_text'] = train['selected_text'].apply(clear_text)
print("\n\n",train['selected_text'])

In [ ]:
# To expand abbrevations like "OMG" it became "Oh My God"

train['text'] = train['text'].apply(abbrev2_text)
print(train['text'])

train['selected_text'] = train['selected_text'].apply(abbrev2_text)
print("\n\n",train['selected_text'])

# Lemmatizer 

In [ ]:
print(train['text'])

In [ ]:
regular_punct = list(string.punctuation)

def spacing_punctuation(text):
    # Add spaces before and after punctuations
    regular_punct = list(string.punctuation)
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text.strip()

In [ ]:
# to space punctuation which helps in tokenization .

train['text'] = train['text'].apply(spacing_punctuation)
print(train['text'])

train['selected_text'] = train['selected_text'].apply(spacing_punctuation)
print("\n\n", train['selected_text'])

# To make sure special characters are handled in word embedding matrix

In [ ]:
# loading the words embedding from glove dictionary to use it later in the model embedding layer

embeddings_index = {}
path = "/kaggle/input/glove6b100dtxt/glove.6B.100d.txt"

f = open(path)
for line in f:
    values = line.split(' ')
    # The first element indicate the word
    word = values[0]      
    # And rest of the part is vector representation of that word.
    coefs = np.asarray(values[1:], dtype='float32') 
    embeddings_index[word] = coefs
f.close()

print('The number of words in GloVe : ', len(embeddings_index))

In [ ]:
regular_punct = list(string.punctuation)
regular_punct

In [ ]:
# cjecking which symbols aren't supported in our embedding dictionary 

all_punct = list(set(regular_punct))
not_supported_symbol = []
for e in all_punct:
    if e not in embeddings_index:
        not_supported_symbol.append(e)
print("Special symbol not present in word embedding : ", not_supported_symbol)

In [ ]:
print(train['text'])


# Converting selected data to its corresponding indices in the original text

In [ ]:
# here we search for the index of the first word of the selected text in the original text and store that index 
# as start index in indices array touse it in the evaluation process

def find_start_indices(x):
    text,sel_text  = x[0],x[1]
    text = text.split()
    sel_text = sel_text.split()
    
    if len(sel_text) == 0 or len(text) == 0:
        return 0
    
    end = sel_text[0]
    if end in text:
        index = text.index(end)
        return index
    else:
        return 0

    
    
train['start_indices'] = train[['text','selected_text']].apply(lambda x:find_start_indices(x),axis=1)


# here we search for the index of the last word of the selected text in the original text and store that index 
# as the end index in indices array touse it in the evaluation process

def find_end_indices(x):
    text,sel_text,start_indices  = x[0],x[1],x[2]
    text = text.split()
    sel_text = sel_text.split()
    
    if len(sel_text) == 0 or len(text) == 0:
        return start_indices
    
    end = sel_text[-1]
    try:
        index = text.index(end,start_indices)
    except:
        if end in text:
            index = text.index(end)
            return index
        else: return start_indices + 1
    return index

train['end_indices'] = train[['text','selected_text','start_indices']].progress_apply(lambda x:find_end_indices(x),axis=1)


train.sample(15)


In [ ]:
# function to return the indices of the sentences with length >= 30 to drop them

def drop_long_sentences(text_list):
    indices = locate(text_list, lambda x: len(x) > 30)
    return list(indices)

In [ ]:
# function to return the indices of the sentences with certain sentiment to drop them

def drop_sentiment(text_list, sentiment):
    indices = locate(text_list, lambda x: x == sentiment)
    return list(indices)

In [ ]:
print("total number of training samples before dropping >> ", train.shape[0], "\n")
train_txt = train['text'].values
tokenizer_text = Tokenizer(lower = True,split = ' ',oov_token = 'oov')
tokenizer_text.fit_on_texts(train_txt)
train_txt = tokenizer_text.texts_to_sequences(train_txt)
max_before = max(len(seq) for seq in train_txt)
indx = drop_long_sentences(train_txt)
text_len_list = [len(seq) for seq in train_txt]
train.drop(train.index[indx], inplace = True)
train_txt = np.delete(train_txt, indx).tolist()
max_text_len = max(len(seq) for seq in train_txt)
print("total number of training samples after dropping >>", len(train_txt), "\n")
print("max length before dropping >> ", max_before, "\n")
print("max length after dropping >> ", max_text_len, "\n")

In [ ]:
unique_len, freq = np.unique(text_len_list, return_counts=True)
print(unique_len, "\n")
print(freq, "\n")
fig, ax = plt.subplots(figsize =(10, 8))
ax.barh(unique_len, freq, color ='maroon')
ax.invert_yaxis()
plt.title("length distribution for training sentences")
plt.show()

In [ ]:
val = [len(train_txt), len(indx)]
lab = ["used data for training", "dropped data"]
plt.pie(val, labels = lab, autopct='%1.2f%%')
plt.legend()
plt.show() 

# This section is only to test the model performance on positive and negative data only so it would drop the neutral samples from our data

In [ ]:

#train_sent = train['sentiment'].values
#ind = drop_sentiment(train_sent, "neutral")
#print(len(ind))
#random.shuffle(ind)



In [ ]:
#train.drop(train.index[ind], inplace=True)
#print(train.shape[0])
#train

In [ ]:
#train.drop(train.index[ind[:3000]], inplace=True)
#print(train.shape[0])
#train

In [ ]:
#train

# Split Training Dataset

In [ ]:
train_labels = train[['start_indices', 'end_indices']]
train_data = train[['text', 'sentiment']]
train_text_data, analysis_data, train_label_data, analysis_label = train_test_split(train_data, train_labels, test_size = 0.2, random_state = 42, shuffle = True)

train_text_data, validation_text, train_label_data, validation_label = train_test_split(train_text_data, train_label_data, test_size = 0.06, random_state = 42, shuffle = True)

print("\n training dataset size", len(train_text_data))
print("\n validation dataset size",len(validation_text))
print("\n dataset used for analysis size",len(analysis_data))

# Preparing Training and validation data with padding to pass it as inputs to the model

In [ ]:
train_text = train_text_data['text'].values
val_text = validation_text['text'].values

tokenizer_text = Tokenizer(lower = True, split = ' ', oov_token = 'oov')
tokenizer_text.fit_on_texts(train_text)


train_text = tokenizer_text.texts_to_sequences(train_text)
val_text = tokenizer_text.texts_to_sequences(val_text)

vocab_size_text = len(tokenizer_text.word_index) + 1

word_index_text = tokenizer_text.word_index

train_text = pad_sequences(train_text, maxlen = max_text_len, padding = 'post')
val_text = pad_sequences(val_text, maxlen = max_text_len, padding = 'post')

print(train_text.shape,val_text.shape)

In [ ]:
train_sentiment = train_text_data['sentiment'].values
val_sentiment = validation_text['sentiment'].values

tokenizer_sentiment = Tokenizer(lower = True, split = ' ', oov_token = 'oov')
tokenizer_sentiment.fit_on_texts(train_sentiment)

train_sentiment = tokenizer_sentiment.texts_to_sequences(train_sentiment)
val_sentiment = tokenizer_sentiment.texts_to_sequences(val_sentiment)

max_sentiment_len = max(len(seq) for seq in train_sentiment)



train_sentiment = pad_sequences(train_sentiment, maxlen = max_sentiment_len, padding = 'post')
val_sentiment = pad_sequences(val_sentiment, maxlen = max_sentiment_len, padding = 'post')

vocab_size_sentiment = len(tokenizer_sentiment.word_index) + 1
word_index_sentiment = tokenizer_sentiment.word_index

print(train_sentiment.shape,val_sentiment.shape)

In [ ]:
val_inp = (val_text, val_sentiment)
val_out = validation_label.values
val_data = (val_inp, val_out)

# Preparing Embedding Matrix for text and sentiment

In [ ]:
# create embedding index
embedding_index = {}
with open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix_text=np.zeros((vocab_size_text, 100))
for word, i in word_index_text.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_text[i]=embedding_vector
 

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix_sentiment=np.zeros((len(word_index_sentiment) + 1, 100))
for word, i in word_index_sentiment.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_sentiment[i]=embedding_vector

In [ ]:
embedding_matrix_text.shape

In [ ]:
embedding_matrix_sentiment.shape

# **Preprocessing Testing dataset**

In [ ]:
test

In [ ]:
# applying preprocessing as training data

test['text'] = test['text'].apply(remove_hyperlinks)
test['text'] = test['text'].apply(clear_text)
test['text'] = test['text'].apply(expand_contractions)
test['text'] = test['text'].apply(abbrev2_text)
#test['text'] = test['text'].apply(spacy_lemmatizer)
test['text'] = test['text'].apply(spacing_punctuation)

print(test['text'])

In [ ]:
# apply padding on test data

test_text = test['text'].values
test_sentiment = test['sentiment'].values

test_text = tokenizer_text.texts_to_sequences(test_text)
test_text = pad_sequences(test_text, maxlen = max_text_len, padding = 'post')

test_sentiment = tokenizer_sentiment.texts_to_sequences(test_sentiment)
test_sentiment = pad_sequences(test_sentiment, maxlen = max_sentiment_len, padding = 'post')


print(test_text.shape, test_sentiment.shape)

# Jaccard Calculation

In [ ]:
def jaccard(str1, str2): 
    
    a = set(str1.lower().split()) 
    
    b = set(str2.lower().split())
    
    c = a.intersection(b)
    
    return float(len(c)) / (len(a) + len(b) - len(c) + 0.0000000000008)

# Building the Model

# GRU MODEL Training 

In [ ]:
def build_model1(max_length_text, max_length_sentiment):
    
    input1=Input(shape=(max_length_text,),name='text_input')
    
    embed = Embedding(vocab_size_text,100,input_length=max_length_text,name='embedding_text',\
                          trainable=False,embeddings_initializer=tf.constant_initializer(embedding_matrix_text))(input1)
    
    gru = GRU(32,name='gru',return_sequences=True)(embed)
    
    f1=Flatten()(gru)
    
    input2=Input(shape=(max_length_sentiment,),name='sentiment_input')
    
    embed2=Embedding(vocab_size_sentiment,100,input_length=max_length_sentiment,name='embedding_sentiment')(input2)
    
    f2=Flatten()(embed2)
    
    concat1=Concatenate(axis=1)([f1,f2])
    
    dense1=Dense(16,activation='relu',kernel_regularizer=l2(0.0001))(concat1)
    
    drop1 = Dropout(0.6)(dense1)
    
    ln= LayerNormalization()(drop1)
    
    dense2=Dense(8,activation='relu',kernel_regularizer=l2(0.0001))(ln)
    
    output=Dense(2,name='output')(dense2)
    
    model = Model(inputs=[input1, input2], outputs=[output])
    
    return model

In [ ]:
model_1 = build_model1(max_text_len, max_sentiment_len)
model_1.summary()

In [ ]:
model_1.compile(
  optimizer= 'adam',
  loss= 'mse',
  metrics= ['accuracy']
)
plot_model(model_1)

In [ ]:
history = model_1.fit(x = (train_text, train_sentiment), y = train_label_data.values, batch_size = 128, epochs = 150, validation_data = val_data, validation_batch_size = 64)

# Bi LSTM model Training 

In [ ]:
def build_model2(max_length_text, max_length_sentiment):
    
    input1 = Input(shape = (max_length_text, ), name = 'text_input')
    
    embed = Embedding(vocab_size_text, 100, input_length = max_length_text,name = 'embedding_text',\
                          trainable = False, embeddings_initializer = tf.constant_initializer(embedding_matrix_text))(input1)
    
    lstm = Bidirectional(LSTM(32, name = 'BLSTM', return_sequences = True))(embed)
    
    f1 = Flatten()(lstm)
    
    input2 = Input(shape = (max_length_sentiment, ), name = 'sentiment_input')
    
    embed2 = Embedding(vocab_size_sentiment, 100, input_length = max_length_sentiment, name = 'embedding_sentiment')(input2)
    
    f2 = Flatten()(embed2)
    
    concat1 = Concatenate(axis = 1)([f1, f2])
    
    dense1 = Dense(16, activation = 'relu',kernel_regularizer = l2(0.0001))(concat1)
    
    drop1 = Dropout(0.6)(dense1)
    
    ln = LayerNormalization()(drop1)
    
    dense2 = Dense(8, activation = 'relu', kernel_regularizer = l2(0.0001))(ln)
    
    output = Dense(2, name = 'output')(dense2)
    
    model = Model(inputs = [input1, input2], outputs = [output])
    
    return model

In [ ]:
model_2 = build_model2(max_text_len, max_sentiment_len)
model_2.summary()

In [ ]:
model_2.compile(
  optimizer= 'adam',
  loss= 'mse',
  metrics= ['accuracy']
)
plot_model(model_2)

In [ ]:
history = model_2.fit(x = (train_text, train_sentiment), y = train_label_data.values, batch_size = 128, epochs = 150, validation_data = val_data, validation_batch_size = 64)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy ', 'validation accuracy'], loc = 'lower right')
plt.title("training accuracy and validation accuracy along epochs")
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')

plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'], loc='upper right')
plt.title("training loss and validation loss along epochs")
plt.show()

# Calculate Training Jaccard Score .

In [ ]:
# load predictions of the training data to apply jaccard matric

training_pred =  model_2.predict([train_text, train_sentiment])
print(len(training_pred))
print(training_pred)

In [ ]:
# convert the output of the model from indices to text by extracting the substring from strat to end index

def find_sel_text(x):
  txt , start , end = x[0], x[1], x[2]

  end = end + 1
  txt = txt.split()
  sel_text = txt[start:end]
  sel_text = " ".join(sel_text)
  return sel_text


In [ ]:
# by rounding the model output to int and sort it we get array that contain ythe start and end indices .

training_pred = np.sort(abs((np.round(training_pred))))

train_text_data['predicted_start'],train_text_data['predicted_end'] = (abs(training_pred[:,0])),(abs(training_pred[:,1]))

train_text_data['predicted_start'] = train_text_data['predicted_start'].astype('int')
train_text_data['predicted_end'] = train_text_data['predicted_end'].astype('int')

train_text_data['predicted_selected_text'] = train_text_data[['text','predicted_start', 'predicted_end']].apply(lambda x:find_sel_text(x),axis=1)
train_text_data['true_start'], train_text_data['true_end'] = train_label_data['start_indices'],train_label_data['end_indices'] 
train_text_data['true_selected_text'] = train_text_data[['text','true_start', 'true_end']].apply(lambda x:find_sel_text(x),axis=1)

train_text_data['jaccard'] = train_text_data.apply(lambda y:jaccard(y.predicted_selected_text, y.true_selected_text), axis = 1)

print("\n training total jaccard score",np.mean(train_text_data['jaccard']), "\n")

print("\n training total jaccard score for positive samples",np.mean(train_text_data[train_text_data['sentiment'] == 'positive']['jaccard']), "\n")

print("\n training total jaccard score for negative samples",np.mean(train_text_data[train_text_data['sentiment'] == 'negative']['jaccard']), "\n")

print("\n training total jaccard score for neutral samples",np.mean(train_text_data[train_text_data['sentiment'] == 'neutral']['jaccard']), "\n")

display(train_text_data[:20])

# Kaggle test dataset prediction

In [ ]:
test_selected_indx  = model_2.predict([test_text,test_sentiment])
print(test_selected_indx.shape)


test_selected_indx = np.round(test_selected_indx)



test['start_indices'],test['end_indices'] = (abs(test_selected_indx[:,0])),(abs(test_selected_indx[:,1]))

test['start_indices'] = test['start_indices'].astype('int')
test['end_indices'] = test['end_indices'].astype('int')



In [ ]:
test['selected_text'] = test[['text','start_indices', 'end_indices']].apply(lambda x:find_sel_text(x),axis=1)

test

# Success & Failure Cases Analysis on analysis (test splitted from training) dataset

In [ ]:
# apllying padding on analysis data (test data) to pass it to the model

an_text = analysis_data['text'].values
an_sentiment = analysis_data['sentiment'].values


an_text = tokenizer_text.texts_to_sequences(an_text)
an_text = pad_sequences(an_text, maxlen = max_text_len, padding = 'post')

an_sentiment = tokenizer_sentiment.texts_to_sequences(an_sentiment)
an_sentiment = pad_sequences(an_sentiment, maxlen = max_sentiment_len, padding = 'post')


print(an_text.shape, an_sentiment.shape)

In [ ]:
# getting the pridected indices array

analysis_selected_indx  = model_2.predict([an_text,an_sentiment])
print(analysis_selected_indx.shape)


analysis_selected_indx = abs(np.round(analysis_selected_indx)).astype('int')
analysis_selected_indx = np.sort(analysis_selected_indx)
print((analysis_selected_indx))

an_label = analysis_label.values


In [ ]:

analysis_list = list(zip(analysis_data['text'], analysis_data['sentiment'], analysis_selected_indx[:, 0], analysis_selected_indx[:, 1], an_label[:, 0], an_label[:, 1]))

analysis_df = pd.DataFrame(analysis_list, columns = ['text', 'sentiment', 'predicted_start', 'predicted_end','true_start', 'true_end'])

analysis_df['predicted_selected_text'] = analysis_df[['text','predicted_start', 'predicted_end']].apply(lambda x:find_sel_text(x),axis=1)

analysis_df['true_selected_text'] = analysis_df[['text','true_start', 'true_end']].apply(lambda x:find_sel_text(x),axis=1)

analysis_df['jaccard'] = analysis_df.apply(lambda y:jaccard(y.predicted_selected_text, y.true_selected_text), axis = 1)

#display(analysis_df)

print("\n analysis data total jaccard score",np.mean(analysis_df['jaccard']), "\n")

print("\n analysis data tion total jaccard score for positive samples",np.mean(analysis_df[analysis_df['sentiment'] == 'positive']['jaccard']), "\n")

print("\n analysis data total jaccard score for negative samples",np.mean(analysis_df[analysis_df['sentiment'] == 'negative']['jaccard']), "\n")
 
print("\n analysis data total jaccard score for neutral samples",np.mean(analysis_df[analysis_df['sentiment'] == 'neutral']['jaccard']), "\n")

display(analysis_df[:20])

In [ ]:
# to classify the predictions to success and failure cases

def isSubset(arr1, arr2):
    if arr1[0] <= arr2[0] and arr1[1] >= arr2[1]:
        return True
    else:
        return False
    
def check_intersection(arr1, arr2):
    if arr1[0] == arr2[0] or arr1[1] == arr2[1]:
        return True
    elif arr1[0] < arr2[0] and arr1[1] < arr2[1] and arr2[0] < arr1[1]:
        return True
    elif arr1[0] > arr2[0] and arr1[1] > arr2[1] and arr1[0] < arr2[1]:
        return True
    elif arr1[0] > arr2[0] and arr1[1] < arr2[1]:
        return True
    else:
        return False

In [ ]:
success_index = []
fail_index = []
fail_with_intersection = []

for i in range(len(analysis_selected_indx)):
    if(np.array_equal(analysis_selected_indx[i], an_label[i])):
      success_index.append(i)
    elif(check_intersection(analysis_selected_indx[i], an_label[i]) or isSubset(analysis_selected_indx[i], an_label[i])):
        fail_with_intersection.append(i)
    else:
      fail_index.append(i)
    
print("\n failed cases length >> ",len(fail_index))
print("\n success cases length >> ",len(success_index))
print("\n intersection cases length >> ",len(fail_with_intersection))



In [ ]:
an_txt = analysis_data['text'].values
an_sen = analysis_data['sentiment'].values

success_text = []
failure_text = []
success_sentiment = []
failure_sentiment = []
success_indices = []
failure_indices = []
success_ground = []
failure_ground = []
intr_fail_text = []
intr_fail_sent = []
intr_fail_ind = []
intr_fail_ground = []

for k in range(len(success_index)):
    success_text.append(an_txt[success_index[k]])
    success_sentiment.append(an_sen[success_index[k]])
    success_indices.append(analysis_selected_indx[success_index[k]]) 
    success_ground.append(an_label[success_index[k]])

for k in range(len(fail_index)):
    failure_text.append(an_txt[fail_index[k]])
    failure_sentiment.append(an_sen[fail_index[k]])
    failure_indices.append(analysis_selected_indx[fail_index[k]]) 
    failure_ground.append(an_label[fail_index[k]])
    
for k in range(len(fail_with_intersection)):
    intr_fail_text.append(an_txt[fail_with_intersection[k]])
    intr_fail_sent.append(an_sen[fail_with_intersection[k]])
    intr_fail_ind.append(analysis_selected_indx[fail_with_intersection[k]]) 
    intr_fail_ground.append(an_label[fail_with_intersection[k]])
    
success_tup = list(zip(success_text, success_sentiment, success_indices, success_ground))  
failure_tup = list(zip(failure_text, failure_sentiment, failure_indices, failure_ground))
intr_fail_tup = list(zip(intr_fail_text, intr_fail_sent, intr_fail_ind, intr_fail_ground))
succ_df = pd.DataFrame(success_tup, columns = ['text', 'sentiment', 'predicted indices', 'ground truth indices'])
fail_df = pd.DataFrame(failure_tup, columns = ['text', 'sentiment', 'predicted indices', 'ground truth indices'])
intr_fail_df = pd.DataFrame(intr_fail_tup, columns = ['text', 'sentiment', 'predicted indices', 'ground truth indices'])
print("\n")
print("          ****************** Some Success Cases ***********************")
display(succ_df.iloc[:20])
print("\n")
print("           ****************** Some Failure Cases with Intersection ***********************")
display(intr_fail_df.iloc[:20])
print("\n")
print("           ****************** Some Failure Cases ***********************")
display(fail_df.iloc[:20])


In [ ]:
succ = tokenizer_text.texts_to_sequences(success_text)
fail = tokenizer_text.texts_to_sequences(failure_text)
succ_fail = tokenizer_text.texts_to_sequences(intr_fail_text)

success_len_list = [len(seq) for seq in succ]
failure_len_list = [len(seq) for seq in fail]
succ_fail_len = [len(seq) for seq in succ_fail]

In [ ]:
# analyzing the lengths of success cases

unique_len, freq = np.unique(success_len_list, return_counts=True)
print(unique_len, "\n")
print(freq, "\n")
fig, ax = plt.subplots(figsize =(10, 8))
ax.barh(unique_len, freq, color ='orange')
ax.invert_yaxis()
plt.title('sentences length distribution for success cases samples')
plt.show()

In [ ]:
# analyzing the lengths of failure cases

unique_len, freq = np.unique(failure_len_list, return_counts=True)
print(unique_len, "\n")
print(freq, "\n")
fig, ax = plt.subplots(figsize =(10, 8))
ax.barh(unique_len, freq)
ax.invert_yaxis()
plt.title('sentences length distribution for failure cases samples')
plt.show()

In [ ]:
# analyzing the lengths of intersection cases

unique_len, freq = np.unique(succ_fail_len, return_counts=True)
print(unique_len, "\n")
print(freq, "\n")
fig, ax = plt.subplots(figsize =(10, 8))
ax.barh(unique_len, freq, color ='green')
ax.invert_yaxis()
plt.title('sentences length distribution for intersection cases samples')
plt.show()

In [ ]:
nat_succ = [success_sentiment[index] for (index, item) in enumerate(success_sentiment) if item == "neutral"]
pos_succ = [success_sentiment[index] for (index, item) in enumerate(success_sentiment) if item == "positive"]
neg_succ = [success_sentiment[index] for (index, item) in enumerate(success_sentiment) if item == "negative"]


In [ ]:
nat_fail = [failure_sentiment[index] for (index, item) in enumerate(failure_sentiment) if item == "neutral"]
pos_fail = [failure_sentiment[index] for (index, item) in enumerate(failure_sentiment) if item == "positive"]
neg_fail = [failure_sentiment[index] for (index, item) in enumerate(failure_sentiment) if item == "negative"]

In [ ]:
suc_fail_neu = [intr_fail_sent[index] for (index, item) in enumerate(intr_fail_sent) if item == "neutral"]
suc_fail_pos = [intr_fail_sent[index] for (index, item) in enumerate(intr_fail_sent) if item == "positive"]
suc_fail_neg = [intr_fail_sent[index] for (index, item) in enumerate(intr_fail_sent) if item == "negative"]

In [ ]:
# plot showing the percentage of each sentiment in success samples

val = [len(nat_succ), len(pos_succ), len(neg_succ)]
lab = ["neutral samples", "positive samples", "negative samples"]
plt.pie(val, labels = lab, autopct='%1.2f%%')
plt.legend(loc = 'upper left')
plt.title('success case')
plt.show() 

In [ ]:
# plot showing the percentage of each sentiment in failure samples

val = [len(nat_fail), len(pos_fail), len(neg_fail)]
lab = ["neutral samples", "positive samples", "negative samples"]
plt.pie(val, labels = lab, autopct='%1.2f%%')
plt.legend(loc = 'upper left')
plt.title('failure case')
plt.show() 

In [ ]:
# plot showing the percentage of each sentiment in intersection samples

val = [len(suc_fail_neu), len(suc_fail_pos), len(suc_fail_neg)]
lab = ["neutral samples", "positive samples", "negative samples"]
plt.pie(val, labels = lab, autopct='%1.2f%%')
plt.legend(loc = 'upper left')
plt.title('intersection case')
plt.show() 

In [ ]:
# plot show the achived jaccard scores and their frequencies

sns.distplot(analysis_df['jaccard'], hist = True, kde = True, 
             color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

plt.title('Density Plot of jaccard scores')
plt.xlabel('jaccard score')
plt.ylabel('Density')

In [ ]:
# plot show the achived jaccard scores for each sentiment and their frequencies

sentiments = ['positive', 'negative', 'neutral']

for sentiment in sentiments:
    # Subset to the airline
    subset = analysis_df[analysis_df['sentiment'] == sentiment]
    
    # Draw the density plot
    sns.distplot(subset['jaccard'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3},
                 label = sentiment)
    
plt.legend(prop={'size': 16}, title = 'sentiment')
plt.title('Density Plot of jaccard scores for each sentimnet')
plt.xlabel('jaccard score')
plt.ylabel('Density')